In [ ]:
import scipy.io
import numpy as np
import networkx as nx
from networkx.algorithms import community
import pandas as pd
import itertools as it
import copy
import re

from notebooks.utils import filter_logile, hex_to_rgb, get_lut, lut_label, titleize, distribution_plot, figures_to_html, plotly_tabulate, NbCache


In [ ]:
stats = scipy.io.loadmat('results/nbs/nbs.mat')["test_stat"]
stats[stats < 3.0] = np.NAN

In [ ]:
G = nx.from_numpy_matrix(stats)
for i, j in G.edges:
    if np.isnan(G.edges[i, j]["weight"]):
        G.remove_edge(i, j)
components = community.greedy_modularity_communities(G, weight="weight")

In [ ]:
order = list(it.chain.from_iterable(components))

In [ ]:
def np_str_join(__delim: str, /, arrays):
    arrays = iter(arrays)
    try:
        first = next(arrays)
    except StopIteration:
        return np.array()
    try:
        second = next(arrays)
    except StopIteration:
        return first
    mod = np.char.mod(f"%s{__delim}", first)
    return np_str_join(__delim, it.chain([np.char.add(mod, second)], arrays))

def autopair_labels(labels, delim = ", "):
    back_iter = lambda matrix: np.moveaxis(matrix, 2, 0)
    return np_str_join(
        delim,
        back_iter(
            np.array(np.meshgrid(labels, labels))
                .T
                .reshape((*stats.shape, 2))
                .astype(np.str_)
        )
    )


In [ ]:
df = (
    pd.concat([
        copy.copy(
            orig := (
                raw := pd.read_csv("resources/brainnetome-regions.csv")
            )
            .assign(
                **dict(
                    raw["Modified cyto-architectonic"]
                    .str.split(r',\s?', expand=True)
                    .rename({0: "Name", 1: "Long Name"}, axis=1)
                    .drop(columns=[2])
                ),
                **dict(
                    raw["Gyrus"]
                    .str.split(r',\s?', expand=True)
                    .rename({0: "Gyrus Abbr", 1: "Gyrus"}, axis=1)
                )
            )
            .drop(
                columns=[
                    "Modified cyto-architectonic",
                    "Left and right hemispheres"
                ]
            )
        )
        .drop(columns=["rh.MNI (X, Y, Z)", "Label ID.R"])
        .rename(columns={"lh.MNI (X,Y,Z)": "MNI", "Label ID.L": "Label ID"})
        .assign(hemisphere="R"),

        orig
        .drop(columns=["lh.MNI (X,Y,Z)", "Label ID.L"])
        .rename(columns={"rh.MNI (X, Y, Z)": "MNI", "Label ID.R": "Label ID"})
        .assign(hemisphere="L")
    ])
    .astype({"Label ID": int})
    .assign(
        Lobe=lambda df: df["Lobe"].str.strip()
    )
    .set_index("Label ID")
    .sort_index()
    .sort_values(["hemisphere", "Lobe", "Gyrus"])
    .reindex(index=np.array(order) + 1)
)

In [ ]:
frontal_idx = df[df["Lobe"] == "Frontal lobe"].index.sort_values() - 1
background = np.full_like(stats, np.NAN)
background[np.ix_(frontal_idx, frontal_idx)] = 1

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

def get_names(df):
    return df["Name"] + "_" + df["hemisphere"].str.lower()

def get_labels(df, fields):
    labels = np.dstack([autopair_labels(df[field], " ➜ ").T for field in fields])
    template = "<br>".join(f"<b>{name}:</b> %{{customdata[{i}]}}" for i, name in enumerate(fields))
    return labels, template

labels, template = get_labels(
    df,
    [
        "hemisphere",
        "Lobe",
        "Gyrus",
        "Long Name",
    ]
)

stat_index = df.index - 1
# fig = px.imshow(
#     stats[np.ix_(stat_index, stat_index)],
#     x=get_names(df),
#     y=get_names(df),
#     width=800,
#     height=800
# )
# fig.update_traces(
#     customdata=labels,
#     hovertemplate="<br>".join([
#         "(%{x}, %{y})",
#         template,
#         "T-value: %{z}",
#     ])
# )

fig = go.Figure()
fig.add_traces([
    # go.Heatmap(
    #     z = background[np.ix_(stat_index, stat_index)],
    #     # hoverinfo="none",
    # ),
    go.Heatmap(
        z = stats[np.ix_(stat_index, stat_index)],
        x=get_names(df),
        y=get_names(df),
        hoverongaps=False,
        customdata=labels,
        hovertemplate="<br>".join([
            "(%{x}, %{y})",
            template,
            "T-value: %{z}",
        ]),
    ),

])
fig.update_layout(
    width=1000,
    height=1000,
    yaxis={
        "autorange": "reversed"
    }
)
figures_to_html([fig], "pages/nbs.html")
# fig.show()